In [1]:
import numpy as np
import tensorflow as tf

In [2]:
tf.enable_eager_execution()

In [3]:
tf.version.VERSION

'1.14.0'

In [4]:
nsamples = 10
data_x = np.random.random((nsamples, 25, 25, 526)).astype(np.float32)
data_y = np.random.random((nsamples, 4)).astype(np.float32)

In [5]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [6]:
tfrecords_filename = 'test.tfrecord'

with tf.io.TFRecordWriter(tfrecords_filename) as writer:
    for x, y in zip(data_x, data_y):        
        # Create an example, which will be a row on the TFRecord file
        # and on the dataset when reading it later.
        example = tf.train.Example(features=tf.train.Features(
            feature={
                'Xx': _int64_feature(x.shape[0]),
                'Xy': _int64_feature(x.shape[1]),
                'Xz': _int64_feature(x.shape[2]),
                'X': _bytes_feature(x.tostring()),
                'Yx': _int64_feature(y.shape[0]),
                'Y': _bytes_feature(y.tostring()),
            }))

        # Write the serialized example
        writer.write(example.SerializeToString())

In [7]:
def decode(serialized_example):
    """Parses an image and label from the given `serialized_example`."""
    example = tf.io.parse_single_example(
        serialized_example,
        features={
            'Xx': tf.io.FixedLenFeature([], tf.int64),
            'Xy': tf.io.FixedLenFeature([], tf.int64),
            'Xz': tf.io.FixedLenFeature([], tf.int64),
            'X': tf.io.FixedLenFeature([], tf.string),
            'Yx': tf.io.FixedLenFeature([], tf.int64),
            'Y': tf.io.FixedLenFeature([], tf.string),
        })
    xx = tf.cast(example['Xx'], tf.int64)
    xy = tf.cast(example['Xy'], tf.int64)
    xz = tf.cast(example['Xz'], tf.int64)
    x = tf.io.decode_raw(example['X'], tf.float32)
    x = tf.reshape(x, (xx, xy, xz))
    yx = tf.cast(example['Yx'], tf.int64)
    y = tf.io.decode_raw(example['Y'], tf.float32)
    y = tf.reshape(y, (yx,))
    return x, y

In [8]:
# Define the input pipeline
dataset = tf.data.TFRecordDataset(tfrecords_filename)
dataset = dataset.map(decode)

In [9]:
for i, (x, y) in enumerate(dataset):
    print(tf.abs(x - data_x[i]).numpy().max())
    print(tf.abs(y - data_y[i]).numpy().max())

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
